In [2]:
# imports
from astropy.io import fits
# import matplotlib.pyplot as plt
import numpy as np
import csv 

In [3]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Get Catalog Data 

In [4]:
# open COSMOS 2016 catalog file and get data 
inf16 = fits.open('D:\IfA REU\Catalogs\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits')
data16 = inf16[1].data
cols16 = inf16[1].columns
inf16.close()

In [5]:
# print(cols16)

In [6]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
cols20 = inf20[1].columns
inf20.close()

In [7]:
# print(cols20)

In [8]:
# get id columns
id16_data = np.array(data16['id_k_uv '])
id20_data = np.array(data20['ID_COSMOS2015'])

In [9]:
print(id16_data)
print(id20_data)

[455942 704286 470092 ... 287869 313813 378385]
[-99 -99 -99 ... -99 -99 -99]


In [10]:
##### get the ID lists from the Donley 2012 selected AGN csv file

# initialize list
id_irAGN_csv = []

# open csv file
with open(path_csv+'\COSMOS2020_DONLEY2012_AGN.csv', newline='') as csv_file : 
    # read file 
    reader = csv.reader(csv_file, delimiter=',', quotechar='"')
    # output file into list for easy access
    for row in reader :
        id_irAGN_csv.append(row)
    #print description
    print(id_irAGN_csv[0])

# save important IDs 
AGNid_2020 = id_irAGN_csv[2]

['This file contains the IDs of Donley et al 2012 selected AGN from the COSMOS 2020 catalog. These lists were generated in COSMOS20_AGN_selection.ipynb. Line 1 is the ID, Line 2 is the ID_COSMOS2015.']


In [11]:
# print(data16[0])

# Match catalogs

In [12]:
# returns the index of the match
def FindMatch(search, dataList) :

    # initialize index
    i = 0
    i_match = -99

    # search list 
    for value in dataList :
        # if the two ids match,
        if(search == value) : 
            i_match = i # save current index 
            break       # stop searching 
        else : 
            # increment index and keep searching 
            i = i + 1

    # returns the list index if there is a match...
    # returns -99 if there is no match.
    return(i_match)

In [13]:
# ##### match ID test

# intitialize lists 
id_matchKey = []
test = [1,2,3]
t1 = [0,2,0]
t2 = [1,0,3]

# for each Donley agn
for id in test : 

    # find the index
    i16 = FindMatch(id, t1)
    i20 = FindMatch(id, t2)

    # ad id and location index to the key list 
    id_matchKey.append([id, i16, i20])

    # progress
    print([id, i16, i20])


[1, -99, 0]
[2, 1, -99]
[3, -99, 2]


In [14]:
# ##### match the given ID to the ID_COSMOS2015 in the 2020 catalog and the id_k_uv in the 2016 catalog

# intitialize list 
id_matchKey = []

# for each Donley agn
for id in AGNid_2020 : 

    # find the index
    i16 = FindMatch(id, id16_data)
    i20 = FindMatch(id, id20_data)
    
    # ad id and location index to the key list 
    id_matchKey.append([id, i16, i20])

    # progress
    print([id, i16, i20])


['215821', -99, -99]
['215405', -99, -99]
['216308', -99, -99]
['218496', -99, -99]
['219599', -99, -99]
['221224', -99, -99]
['221280', -99, -99]
['221793', -99, -99]
['221933', -99, -99]
['222912', -99, -99]
['223559', -99, -99]
['223756', -99, -99]
['223968', -99, -99]
['224665', -99, -99]
['225931', -99, -99]
['226195', -99, -99]
['225978', -99, -99]
['226837', -99, -99]
['226472', -99, -99]
['228653', -99, -99]
['230426', -99, -99]
['234115', -99, -99]
['234745', -99, -99]


KeyboardInterrupt: 

In [ ]:
listTest = []

listTest.append( [1,2,3])
listTest.append( ['a', 'b', 'c'])

print(listTest)
print(listTest[1][1])



# TO DO

1. get "ID_CHANDRA=" AGN IR list from COSMOS 2020 
2. match 2020 to 2016 catalog (get IRAC data)
3. select AGN using XRAYS
4. get list of ids
5. match IR and XRAY IDs
6. Plot IR and XRAY selected AGN on same plot using IRAC filters (i.e. IR blue dot, XRAY red x)

